# Neural network

- Combine multiple non-linear combinations (by using non-linear activations for linear combinations) to represent **complex/non-linear behaviors**


Deep learning: a constructing networks of parameterized functional (differentiable) modules. Training a DL model = training these networks from examples (data) using a gradient-based optimization

Universal approximation theorem of NN: neural networks with a single hidden layer can be used to approximate any continuous function to any desired precision

Proof of Universal approximation theorem using interactive visual graph: http://neuralnetworksanddeeplearning.com/chap4.html

In short: the basic trick is that he shows that with a few different numbers, you can basically cause these things to create little boxes (or jump functions). These boxes are adjustable in height, size and location, and you can even combined them => you can use these boxes as basic elements to approximately construct any functions, in a shape of connection of towers

## Tip for training neural net

1. Try to overfit by using a complex model with no regularization such as dropout, augmentation, L1 L2 ...
2. Reduce overfit by:
    - Add data (real data)
    - Data augmentation (add augmented data)
    - Change architectures that generalize well
    - Add regularization (dropout, wd)
    - Reduce architecture complexity

## Activation functions and their consequences

**Sigmoid**:
<img src="images/sigmoid-func.png">

Pros: 
- **when you want your activation to be between 0 and 1** (probability for binary classification, but you should use softmax instead)
- **Good for multilabel classification**: softmax is not suitable as softmax is designed to differentiate ONLY ONE VALUE from another, so softmax is best used for multiclass one-label-each classification. On another hand, sigmoid treats every outputs independently from each other.

Cons: 
- Between -1 and 1, sigmoid tends to be just a linear line => there's no non-linear transformation. Be cautious about this since activation functions are most often perfomed AFTER batchnorm, so batchnorm output should not forced data to have mean 0 and std 1
- Cause **vanishing gradient**: as part of backprop, grad_wrp_input (which is cumulated through multiple layers for layers' weight update) will be multiplied by *activation function's gradient*. **Sigmoid grad maxed out at 0.25 and majority of grads are 0 at 2 tails, so at its best, grad_wrp_input lost by 75%, and at its worst, lots of grad_wrt_input is 0**

**RELU**:
<img src="images/relu-func.png">

Pros:
- Fix vanishing gradient problem caused by sigmoid

Cons:
- There is still a chance for vanishing gradient, as grad_wrt_input will be 0 when input to RELU is negative
    - Solution: use leaky relu, where leaky relu's grad is either 1 (for positive input) or alpha (for negative input), typically 0.2 

- RELU is prone to **exploding gradient**, since the RELU outputs are not capped, so they can be large
    - Solution: do batchnorm
    - Solution 2: switch to weight initialization that is specifically designed for RELU (Kaiming He)

## Weight initialization

Weights are used during forward pass (X@weight + b) and backward pass (matmul with grad_wrt_input), so a good set of initial values for weights are crucial. **And we want our activation to have mean 0 and std 1**


## Xavier init

```python

n_hidden = 50
w1 = torch.randn(m,n_hidden)/math.sqrt(m)
b1 = torch.zeros(n_hidden)
```

Why did we have to divide w1 by sqrt(# of inp features)? 

- Simple explanation: Asume # of inp features is 512. We sum 512 of those things that have a mean of zero, **and a variance of 1**, so if we repeat that 512 times (as y=a@x  will have shape (512,1)), we get something that has a mean of 0, and variance of 512. (See maths and experiments in part2_2019_note.docx)

However, Xavier init does not work well with RELU, which technically discard half of our data by setting them to 0. Thus using RELU activation, we just ignore our effort to make activation have mean 0 and std 1

## Kaiming He

```python

w1 = torch.randn(m,n_hidden)*math.sqrt(2/m) # mean 0, std: sqrt(2/# layers)
b1 = torch.zeros(n_hidden)

```

(See maths and experiments in part2_2019_note.docx

## Why do we obsessed with mean 0 and std 1 for our activation?

In another words: why do we need to normalized data?

- (taken from kaggle_competition_steps.docx), for data that ranges differently across features, data points are spread around and **form an ellipsoid** => eclipse convex functions => **weight update step won't easily reach local optima

- However, if the features are normalized they will be more concentrated and hopefully, **form a unit circle and make the covariance diagonal or at least close to diagonal** => nice and even convex functions => **weight update step can reach local optima easier** . This is what the idea is behind methods such as batch-normalizing the intermediate representations of data in neural networks

- Also, without proper scaling (mean 0 std 1), some input X can be extremely large or small, especially after few layers of neural net, even if your weights are properly initialized
    - Some activations are high and saturate model => cause large gradients and influence the training too much => overfitting => model will train poorly
    - With extremely large/small inputs => vanishing/exploding gradients

## Batchnorm

As mentioned above, Batch normalization (batchnorm) is a way to ensure that activations don't become too high or too low at any point in the model.

Batchnorm exists to:
- ensure mean and variance of input layer/hidden layer stay the same
- make the **neural net output approximately invariant to the scale of the activations of the previous layers (scale invariance)**
- be able to control mean and variance during the course of training: model will learn its own mean and variance for each layer

Note: the idea that batchnorm fix covariance shift problem (previous layers will affect following layer since previous layer's outputs change all the time due to weight updating, thus make the data distribution shifted slightly => model will train poorly) might not be true anymore, since covariance shift itself might not be actually true.

=> **Batchnorm actually smoothens the loss landscape, which explain why deep learning model can train very fast and efficient with batchnorm implementation**

For implementation-from-scratch of batchnorm, look at pseudo code at fastai_words/note.docx, or actual implementation at https://www.coursera.org/learn/deep-neural-network/discussions/weeks/3/threads/6DCRSjwqEeifWAoEAy-2pg

## L1 vs L2 regularizations

(look at fastai_words/note.docx for graph)

The idea of both of these are to **reduce the weight that the function carry by adding those weights to the cost to increase the cost**; as some weights gradients are relative big/small => some weights are being updated faster, some are updated prettly slow => train poorly

- L2: add (lamda * sum of squared weights) to cost, thus limit their growth: **L2 will continuously squeeze model weights and force them to be small (that's why it is also called weight decay)** => some important weights, however, will consistently get back up, while less important weights (due to noise which can cause overfitting) will remain smaller
    - The optimal point is a point which is common between concentric ellipse (from loss landscape) and a circle (penalty term or L2 term) (look at fastai_words/note.docx for graph)
    - L2 forces neural net to distribute learning across ALL WEIGHTS

- L1: add (lambda * sum absolute weights) as regularization term
    - Shape of penalty term is diamond (not a circle)
    - High chances that the loss landscape and the diamond will intersect at the corner point of the diamon, which makes some of the weights 0 => end up with a sparse coefficents => from this, you can utilize L1 to find feature importance

## Batchnorm vs L2 (for SGD)

Since the idea of L2 (or weight decay) is that: important weights should experience gradients to restore their magnitudes affected by weight decay, and less important weights (that try to fit noise) would decay away, batchnorm will **undo these changes** since all weights will be updated equally...

Therefore,  **weigtht decay no longer does any regularizawtion of layers that comes before batchnorm layer**